In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1317563,2021-05-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1317564,2021-05-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1317565,2021-05-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1317566,2021-05-31,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
34995,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34997,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34999,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35001,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35003,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
808893,2021-05-28,Arkansas,Arkansas,5001,2123,37.00,5000,Arkansas,AR,Arkansas,State,3017804
808895,2021-05-29,Arkansas,Arkansas,5001,2124,37.00,5000,Arkansas,AR,Arkansas,State,3017804
808897,2021-05-30,Arkansas,Arkansas,5001,2124,37.00,5000,Arkansas,AR,Arkansas,State,3017804
808899,2021-05-31,Arkansas,Arkansas,5001,2125,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1317563,2021-05-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1317564,2021-05-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1317565,2021-05-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1317566,2021-05-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-06-01') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
497,2021-06-01,Snohomish,Washington,38954,585.00,53061,WA,County,822083,4738.45,71.16
992,2021-06-01,Cook,Illinois,553926,10882.00,17031,IL,County,5150233,10755.36,211.29
1486,2021-06-01,Orange,California,272156,5065.00,6059,CA,County,3175692,8569.97,159.49
1979,2021-06-01,Maricopa,Arizona,550602,10071.00,4013,AZ,County,4485414,12275.39,224.53
2472,2021-06-01,Los Angeles,California,1244831,24364.00,6037,CA,County,10039107,12399.82,242.69
...,...,...,...,...,...,...,...,...,...,...,...
1316763,2021-06-01,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08
1316995,2021-06-01,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1317196,2021-06-01,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1317393,2021-06-01,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
497,2021-06-01,Snohomish,Washington,38954,585.00,53061,WA,County,822083,4738.45,71.16,71.16,4738.45
992,2021-06-01,Cook,Illinois,553926,10882.00,17031,IL,County,5150233,10755.36,211.29,211.29,10755.36
1486,2021-06-01,Orange,California,272156,5065.00,6059,CA,County,3175692,8569.97,159.49,159.49,8569.97
1979,2021-06-01,Maricopa,Arizona,550602,10071.00,4013,AZ,County,4485414,12275.39,224.53,224.53,12275.39
2472,2021-06-01,Los Angeles,California,1244831,24364.00,6037,CA,County,10039107,12399.82,242.69,242.69,12399.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316763,2021-06-01,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08,75.08,2552.55
1316995,2021-06-01,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1317196,2021-06-01,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1317393,2021-06-01,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
845878,2021-06-01,Hale,Alabama,2254,78.00,1065,AL,County,14651,15384.62,532.39,532.39,15384.62,1
920873,2021-06-01,Clarke,Alabama,3526,61.00,1025,AL,County,23622,14926.76,258.23,258.23,14926.76,2
661581,2021-06-01,Lowndes,Alabama,1412,54.00,1085,AL,County,9726,14517.79,555.21,555.21,14517.79,3
586505,2021-06-01,Etowah,Alabama,14013,359.00,1055,AL,County,102268,13702.23,351.04,351.04,13702.23,4
492036,2021-06-01,Franklin,Alabama,4278,82.00,1059,AL,County,31362,13640.71,261.46,261.46,13640.71,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247532,2021-06-01,Platte,Wyoming,651,11.00,56031,WY,County,8393,7756.46,131.06,131.06,7756.46,19
807180,2021-06-01,Converse,Wyoming,1013,17.00,56009,WY,County,13822,7328.90,122.99,122.99,7328.90,20
1003945,2021-06-01,Lincoln,Wyoming,1413,12.00,56023,WY,County,19830,7125.57,60.51,60.51,7125.57,21
1117701,2021-06-01,Niobrara,Wyoming,162,2.00,56027,WY,County,2356,6876.06,84.89,84.89,6876.06,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
661581,2021-06-01,Lowndes,Alabama,1412,54.00,1085,AL,County,9726,14517.79,555.21,555.21,14517.79,3,1
845878,2021-06-01,Hale,Alabama,2254,78.00,1065,AL,County,14651,15384.62,532.39,532.39,15384.62,1,2
281078,2021-06-01,Walker,Alabama,7296,278.00,1127,AL,County,63521,11485.97,437.65,437.65,11485.97,28,3
584769,2021-06-01,Clay,Alabama,1581,57.00,1027,AL,County,13235,11945.60,430.68,430.68,11945.60,20,4
713538,2021-06-01,Greene,Alabama,930,34.00,1063,AL,County,8111,11465.91,419.18,419.18,11465.91,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711809,2021-06-01,Hot Springs,Wyoming,370,3.00,56017,WY,County,4413,8384.32,67.98,67.98,8384.32,17,19
978870,2021-06-01,Uinta,Wyoming,2266,13.00,56041,WY,County,20226,11203.40,64.27,64.27,11203.40,5,20
1003945,2021-06-01,Lincoln,Wyoming,1413,12.00,56023,WY,County,19830,7125.57,60.51,60.51,7125.57,21,21
658983,2021-06-01,Albany,Wyoming,4516,15.00,56001,WY,County,38880,11615.23,38.58,38.58,11615.23,4,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4738.45,20,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4738.45,20,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4738.45,20,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4738.45,20,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4738.45,20,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314483,2021-05-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1314484,2021-05-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1314485,2021-05-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1314486,2021-05-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
842370,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15384.62,2,1,1.00
842371,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15384.62,2,1,0.00
842372,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15384.62,2,1,0.00
842373,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15384.62,2,1,0.00
842374,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15384.62,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020789,2021-05-28,Crook,Wyoming,446,12.00,56011,WY,County,7584,5880.80,158.23,158.23,5880.80,6,23,4.00
1020790,2021-05-29,Crook,Wyoming,446,12.00,56011,WY,County,7584,5880.80,158.23,158.23,5880.80,6,23,0.00
1020791,2021-05-30,Crook,Wyoming,446,12.00,56011,WY,County,7584,5880.80,158.23,158.23,5880.80,6,23,0.00
1020792,2021-05-31,Crook,Wyoming,446,12.00,56011,WY,County,7584,5880.80,158.23,158.23,5880.80,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
658931,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,1.00,0.00
658932,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
658933,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
658934,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
658935,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277970,2021-05-28,Teton,Wyoming,3784,9.00,56039,WY,County,23464,16126.83,38.36,38.36,16126.83,23,1,3.00,0.00
277971,2021-05-29,Teton,Wyoming,3784,9.00,56039,WY,County,23464,16126.83,38.36,38.36,16126.83,23,1,0.00,0.00
277972,2021-05-30,Teton,Wyoming,3784,9.00,56039,WY,County,23464,16126.83,38.36,38.36,16126.83,23,1,0.00,0.00
277973,2021-05-31,Teton,Wyoming,3784,9.00,56039,WY,County,23464,16126.83,38.36,38.36,16126.83,23,1,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-06-01') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
335606,2021-06-01,Imperial,California,28732,735.00,6025,CA,County,181215,15855.20,405.60,405.60,15855.20,1,2,0.00,0.00
2472,2021-06-01,Los Angeles,California,1244831,24364.00,6037,CA,County,10039107,12399.82,242.69,242.69,12399.82,2,5,109.00,4.00
166984,2021-06-01,San Bernardino,California,298545,4758.00,6071,CA,County,2180085,13694.19,218.25,218.25,13694.19,3,4,256.00,0.00
588193,2021-06-01,Inyo,California,1430,38.00,6027,CA,County,18039,7927.27,210.65,210.65,7927.27,4,25,0.00,0.00
74609,2021-06-01,Stanislaus,California,62842,1066.00,6099,CA,County,550660,11412.12,193.59,193.59,11412.12,5,9,54.00,0.00
34542,2021-06-01,Riverside,California,300732,4608.00,6065,CA,County,2470546,12172.69,186.52,186.52,12172.69,6,7,31.00,0.00
59345,2021-06-01,San Joaquin,California,73899,1419.00,6077,CA,County,762148,9696.15,186.18,186.18,9696.15,7,17,105.00,0.00
92526,2021-06-01,Tulare,California,49918,846.00,6107,CA,County,466195,10707.54,181.47,181.47,10707.54,8,10,-16.00,0.00
33638,2021-06-01,Fresno,California,102507,1706.00,6019,CA,County,999101,10259.92,170.75,170.75,10259.92,9,14,8.00,0.00
532965,2021-06-01,Merced,California,32168,470.00,6047,CA,County,277680,11584.56,169.26,169.26,11584.56,10,8,26.00,1.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1247083,2021-06-01,Lassen,California,5736,24.00,6035,CA,County,30573,18761.65,78.50,78.50,18761.65,35,1,0.00,0.00
335606,2021-06-01,Imperial,California,28732,735.00,6025,CA,County,181215,15855.20,405.60,405.60,15855.20,1,2,0.00,0.00
713480,2021-06-01,Kings,California,23076,247.00,6031,CA,County,152940,15088.27,161.50,161.50,15088.27,11,3,-1.00,0.00
166984,2021-06-01,San Bernardino,California,298545,4758.00,6071,CA,County,2180085,13694.19,218.25,218.25,13694.19,3,4,256.00,0.00
2472,2021-06-01,Los Angeles,California,1244831,24364.00,6037,CA,County,10039107,12399.82,242.69,242.69,12399.82,2,5,109.00,4.00
209997,2021-06-01,Kern,California,110147,1389.00,6029,CA,County,900202,12235.81,154.30,154.30,12235.81,14,6,16.00,0.00
34542,2021-06-01,Riverside,California,300732,4608.00,6065,CA,County,2470546,12172.69,186.52,186.52,12172.69,6,7,31.00,0.00
532965,2021-06-01,Merced,California,32168,470.00,6047,CA,County,277680,11584.56,169.26,169.26,11584.56,10,8,26.00,1.00
74609,2021-06-01,Stanislaus,California,62842,1066.00,6099,CA,County,550660,11412.12,193.59,193.59,11412.12,5,9,54.00,0.00
92526,2021-06-01,Tulare,California,49918,846.00,6107,CA,County,466195,10707.54,181.47,181.47,10707.54,8,10,-16.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
335606,2021-06-01,Imperial,California,28732,735.00,6025,CA,County,181215,15855.20,405.60,405.60,15855.20,1,2,0.00,0.00
2472,2021-06-01,Los Angeles,California,1244831,24364.00,6037,CA,County,10039107,12399.82,242.69,242.69,12399.82,2,5,109.00,4.00
166984,2021-06-01,San Bernardino,California,298545,4758.00,6071,CA,County,2180085,13694.19,218.25,218.25,13694.19,3,4,256.00,0.00
588193,2021-06-01,Inyo,California,1430,38.00,6027,CA,County,18039,7927.27,210.65,210.65,7927.27,4,25,0.00,0.00
74609,2021-06-01,Stanislaus,California,62842,1066.00,6099,CA,County,550660,11412.12,193.59,193.59,11412.12,5,9,54.00,0.00
34542,2021-06-01,Riverside,California,300732,4608.00,6065,CA,County,2470546,12172.69,186.52,186.52,12172.69,6,7,31.00,0.00
59345,2021-06-01,San Joaquin,California,73899,1419.00,6077,CA,County,762148,9696.15,186.18,186.18,9696.15,7,17,105.00,0.00
92526,2021-06-01,Tulare,California,49918,846.00,6107,CA,County,466195,10707.54,181.47,181.47,10707.54,8,10,-16.00,0.00
33638,2021-06-01,Fresno,California,102507,1706.00,6019,CA,County,999101,10259.92,170.75,170.75,10259.92,9,14,8.00,0.00
532965,2021-06-01,Merced,California,32168,470.00,6047,CA,County,277680,11584.56,169.26,169.26,11584.56,10,8,26.00,1.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,405.60,15855.20,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,405.60,15855.20,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,405.60,15855.20,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,405.60,15855.20,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,405.60,15855.20,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4402,05/28/21,Lassen,5736,24.00,18761.65,78.50,78.50,18761.65,35,1,0.00,0.00
4403,05/29/21,Lassen,5736,24.00,18761.65,78.50,78.50,18761.65,35,1,0.00,0.00
4404,05/30/21,Lassen,5736,24.00,18761.65,78.50,78.50,18761.65,35,1,0.00,0.00
4405,05/31/21,Lassen,5736,24.00,18761.65,78.50,78.50,18761.65,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,405.60,15855.20,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,405.60,15855.20,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,405.60,15855.20,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,405.60,15855.20,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,405.60,15855.20,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4488,05/28/21,Merced,32142,469.00,11575.19,168.90,169.26,11584.56,10,8,17.00,1.00
4489,05/29/21,Merced,32142,469.00,11575.19,168.90,169.26,11584.56,10,8,0.00,0.00
4490,05/30/21,Merced,32142,469.00,11575.19,168.90,169.26,11584.56,10,8,0.00,0.00
4491,05/31/21,Merced,32142,469.00,11575.19,168.90,169.26,11584.56,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)